Usage of the package
========

First, some imports

In [12]:
import gzip
import pickle
import random
from collections import namedtuple

from territories import Territory, MissingTreeCache

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size. That is why, by default, once created, the tree is stored on disk.

Here we will create a very simple tree out of the **tree.txt** file.

In [2]:
Node = namedtuple('Node', ('id', 'parent_id', 'label', 'level'))
split = lambda x: (arg if arg != 'null' else None for arg in x[:-1].split('; '))

try:
    Territory.load_tree()
except MissingTreeCache:
    with open("tree.txt", "r") as file:
        lines = file.readlines()
        stream = ([Node(*split(x) )for x in lines])
        Territory.build_tree(data_stream=stream, save_tree=False)

Then, you can start to create territories from arbitrary territoiral units.

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [3]:
# some node of the tree
print('\n'.join([f"{e.name} | {e.partition_type}" for e in random.sample(Territory.tree.nodes(), 12)]))

Domfront en Poiraie | COM
Champagnat | COM
Avapessa | COM
Paizay-Naudouin-Embourie | COM
Anglars-Saint-Félix | COM
Reignac | COM
Les Thilliers-en-Vexin | COM
Rouffange | COM
Janville-en-Beauce | COM
Soultz-sous-Forêts | COM
Anost | COM
La Neuville-aux-Joûtes | COM


In [4]:
territory = Territory.from_name()

In [23]:
Territory.tree.nodes()

AttributeError: 'NoneType' object has no attribute 'nodes'

In [28]:
with open("tree_large.txt", "r") as file:
    lines = file.readlines()
    # print(lines)
    with open("tree_large.gzip", "wb") as file:
        file.write(gzip.compress(pickle.dumps(lines)))
        
stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream, save_tree=False)

AttributeError: 'Node' object has no attribute 'tree_id'

In [ ]:
with open("tree_large.gzip", "rb") as file:
    lines = pickle.loads(gzip.decompress(file.read()))

stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream, save_tree=False)

a = Territory.from_name("COM:", "COM:", "DEP:")
b = Territory.from_name("COM:", "COM:", "DEP:")
c = Territory.from_name("COM:", "COM:", "DEP:")
d = Territory.from_name("COM:", "COM:", "DEP:")
e = Territory.from_name("COM:", "COM:", "DEP:")
f = Territory.from_name("COM:", "COM:", "DEP:")

## Operations on entities

Usual operation on territories works as expected :

In [6]:
# addition

print(a, c)
print(a + c) # This simplify to France

Marseille|Saint Etienne Grand Lyon|Île-de-France
France


In [7]:
# substraction

print(a, d)
print(a - d) # only Saint Etienne remains

Marseille|Saint Etienne Marseille|Grand Lyon
Saint Etienne


More importantly, sets operations are also supported

In [8]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Marseille|Saint Etienne and Marseille|Grand Lyon is Marseille
Union of Grand Lyon|Île-de-France and Marseille|Grand Lyon|Île-de-France is Marseille|Grand Lyon|Île-de-France


Territorial units may have parents or children, but Territory do not. As a territory may be formed of several territorial units, it has a LCA, a Lowest Common Ancestor.

In [ ]:
parent = territory.lowest_common_ancestor()